##Install requirements packages

In [1]:
from torch.utils.data import DataLoader
from vncorenlp import VnCoreNLP
from transformers import AutoModel, AutoTokenizer
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.utilities.seed import seed_everything
# from datasets import SentimentDataset
from model import GRUClassifier
# from model_textcnn import TextClassifier



import tensorflow as tf
import pandas as pd 
import numpy as np
from string import digits
from collections import Counter
from pyvi import ViTokenizer
from gensim.models.word2vec import Word2Vec
from keras.utils.np_utils import to_categorical
import os 
from vncorenlp import VnCoreNLP

2022-05-01 10:35:39.965376: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-01 10:35:39.965411: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
rdrsegmenter = VnCoreNLP("./vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 
PATH = ""
data_train = pd.read_csv(PATH+"dataset/vlsp_sentiment_train.csv", sep='\t')
data_train.columns =['Class', 'Data']
data_test = pd.read_csv(PATH+"dataset/vlsp_sentiment_test.csv", sep='\t')
data_test.columns =['Class', 'Data']


In [3]:
import bogo
import regex as re
import itertools
def map_to_unicode():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic

def covert_unicode(txt):
    dicchar = map_to_unicode()
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)
  
def preprocess(document):
    if type(document) is not str:
      document = str(document)
    # convert to lower case
    document = document.lower()

    #remove
  
    # remove html character
    document = re.sub(r'<[^>]*>', '', document)

    #convert all char to unicode
    document = covert_unicode(document)

    #remove adjacent identical characters
    document = ''.join(c[0] for c in itertools.groupby(document))

    #uwf=>ừ,....
    document = bogo.process_sequence(document)

    # remove error character
    document = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',document)

    # remove multiple space character
    document = re.sub(r'\s+', ' ', document).strip()

    return document


In [4]:
data_train['Data'] = data_train['Data'].map(lambda x: preprocess(x))
data_test['Data'] = data_test['Data'].map(lambda x: preprocess(x))

In [5]:

import json
import re
import itertools
import unidecode
import bogo

def read_file(file_path):
    fi = open(file_path, 'r', encoding='utf-8')
    ls = fi.readlines()
    return ls

'''load dictionary to mapping word'''
vowel_file = open(PATH+'correct_teencode_stopword/vietnamese_vowel.json', encoding='utf-8')
vowel_dic = json.load(vowel_file)

short_word_file = open(PATH+'correct_teencode_stopword/short_word.json', encoding='utf-8')
short_word_dic = json.load(short_word_file)

single_word_dic = read_file(PATH+'correct_teencode_stopword/unidecode_vietnamese_dic.txt')
single_word_dic = [re.sub('\n','', s) for s in single_word_dic]


In [6]:
def replace_one_one(word, dictionary):
    new_word = dictionary.get(word,word)
    return new_word
def correct_vowel(sent, vowel_dictionary):
    '''mapping a`, a\ --> à, ....'''
    words = sent.split()
    pattern = r'[aăâeêuưiyoôơ][.`~?\']'
    sent = ""
    for word in words:
        p = re.search(pattern, word)
        new_word = word
        if p:
            idx = p.span()
            replace_vowel = vowel_dictionary[word[idx[0]]][word[idx[0] + 1]]
            new_word = re.sub(pattern, replace_vowel, new_word)
        sent += new_word + ' '
    return sent

def correct_teencode(sent):
    sent = preprocess(sent)
    sent = correct_vowel(sent, vowel_dic)

    words = sent.split()
    sent = ""
    for word in words:
        new_word = ""
        if word[-1] in [',',';']:
            new_word = replace_one_one(word[:-1], short_word_dic)
            sent += new_word + word[-1]
        else:
            new_word = replace_one_one(word, short_word_dic)
            sent += new_word
        sent += ' '
    return sent[:-1]



In [7]:
import copy
data_fixteencode = copy.deepcopy(data_train)
data_fixteencode['Data'] = data_fixteencode['Data'].map(lambda x: correct_teencode(x))

data_train = pd.concat([data_fixteencode, data_train])
# data_train.shape

data_train = data_train.drop_duplicates()
data_train = data_train.dropna()
data_train['Class'].value_counts()


y_train = data_train.iloc[:, 0].values
X_train = data_train.iloc[:, 1].values
y_train = y_train + 1

# new_reviews = []
# for idx,data in enumerate(X_train):
#     text = rdrsegmenter.tokenize(data)
#     text = ' '.join([' '.join(x) for x in text])
#     new_reviews.append(text)
# X_train = new_reviews

# y_val = data_test.iloc[:, 0].values
# X_val = data_test.iloc[:, 1].values
# y_val = y_val + 1


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.4)


In [9]:
from torch.utils.data import Dataset

class SentimentDataset(Dataset):
    def __init__(self, data, labels, transform=None, have_label=True) -> None:
        self.data = data
        self.labels = labels
        self.transform = transform
        self.have_label = have_label

    def __getitem__(self, index):
        sample = self.data[index]
        label = self.labels[index]

        if self.transform:
            sample = self.transform(sample)
        if self.have_label:
            # print(label)
            return sample.lower(), label
        return sample.lower()

    def __len__(self):
        return len(self.data)



In [10]:
train_dataset = SentimentDataset(X_train, y_train)
val_dataset = SentimentDataset(X_val, y_val)


In [ ]:
word_segmentor = VnCoreNLP("vncorenlp/VnCoreNLP-1.1.1.jar", port=6701,
                           annotators="wseg", max_heap_size='-Xmx500m', quiet=True)
# encoder = AutoModel.from_pretrained("phobert_base")
# tokenizer = AutoTokenizer.from_pretrained("phobert_base", use_fast=False)


encoder = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

# encoder = AutoModel.from_pretrained("phobert_large")
# tokenizer = AutoTokenizer.from_pretrained("phobert_large", use_fast=False)
max_sequence_length = 90
output_dim = 3
threshold = 0.5
cp_path = 'lightning_logs/version_155/checkpoints/epoch=26-step=728.ckpt'
model = GRUClassifier.load_from_checkpoint(cp_path,
                                           segmentor=word_segmentor, tokenizer=tokenizer, encoder=encoder,
                                           max_sequence_length=256, output_dim=output_dim, embedding_size=768,
                                           bidirectional=True, n_layers=2, threshold=threshold)
# model = GRUClassifier(word_segmentor, tokenizer, encoder,
#                       max_sequence_length=max_sequence_length, output_dim=output_dim, bidirectional=True,
#                       embedding_size=768, n_layers=2)
# model = TextClassifier(word_segmentor, tokenizer, encoder, max_sequence_length)

# dataset
# train_path = 'data/vieon_comments/train_split_rm_emoji.json'
# train_dataset = SentimentDataset(train_path)
# val_path = 'data/vieon_comments/val_split_rm_emoji.json'
# val_dataset = SentimentDataset(val_path)

# dataloader
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32)
callbacks = [EarlyStopping(
    monitor="val_acc", patience=10, verbose=True, mode="max")]

trainer = pl.Trainer(gpus=[4], precision=16,
                     auto_select_gpus=False, callbacks=callbacks)
trainer.fit(model, train_dataloader, val_dataloader)


Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Using 16bit native Automatic Mixed Precision (AMP)
/root/miniconda3/envs/sentiment/lib/python3.7/site-packages/pytorch_lightning/loops/utilities.py:9

Sanity Checking: 0it [00:00, ?it/s]

/root/miniconda3/envs/sentiment/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


F1 score: 0.28125
-------------------------------
F1 score keras: 0.15000000000000002
ACC score keras: 0.28125
-------------------------------
F1 score: 0.296875
-------------------------------
F1 score keras: 0.19820843570843572
ACC score keras: 0.296875
-------------------------------


/root/miniconda3/envs/sentiment/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


...

F1 score: 0.625
-------------------------------
F1 score keras: 0.6098541980894923
ACC score keras: 0.625
-------------------------------
F1 score: 0.703125
-------------------------------
F1 score keras: 0.6887097787833083
ACC score keras: 0.703125
-------------------------------
F1 score: 0.7604166666666666
-------------------------------
F1 score keras: 0.7481237081727278
ACC score keras: 0.7604166666666666
-------------------------------
F1 score: 0.734375
-------------------------------
F1 score keras: 0.7224887925255573
ACC score keras: 0.734375
-------------------------------
F1 score: 0.7375
-------------------------------
F1 score keras: 0.7289001249295367
ACC score keras: 0.7375
-------------------------------
F1 score: 0.7395833333333334
-------------------------------
F1 score keras: 0.7309125986182679
ACC score keras: 0.7395833333333334
-------------------------------
F1 score: 0.7321428571428571
-------------------------------
F1 score keras: 0.7201473067521661
ACC score 

F1 score: 0.7471064814814815
-------------------------------
F1 score keras: 0.7397903586376366
ACC score keras: 0.7471064814814815
-------------------------------
F1 score: 0.7454545454545455
-------------------------------
F1 score keras: 0.7385222301177394
ACC score keras: 0.7454545454545455
-------------------------------
F1 score: 0.7438616071428571
-------------------------------
F1 score keras: 0.737092442192069
ACC score keras: 0.7438616071428571
-------------------------------
F1 score: 0.7445175438596491
-------------------------------
F1 score keras: 0.7377675430701004
ACC score keras: 0.7445175438596491
-------------------------------
F1 score: 0.7467672413793104
-------------------------------
F1 score keras: 0.740181512634026
ACC score keras: 0.7467672413793104
-------------------------------
F1 score: 0.7447033898305084
-------------------------------
F1 score keras: 0.7383587261964347
ACC score keras: 0.7447033898305084
-------------------------------
F1 score: 0.742708

Validation: 0it [00:00, ?it/s]

F1 score: 0.65625
-------------------------------
F1 score keras: 0.6405723905723906
ACC score keras: 0.65625
-------------------------------
F1 score: 0.75
-------------------------------
F1 score keras: 0.7282547699214366
ACC score keras: 0.75
-------------------------------
F1 score: 0.8020833333333334
-------------------------------
F1 score keras: 0.7853906563056889
ACC score keras: 0.8020833333333334
-------------------------------
F1 score: 0.7578125
-------------------------------
F1 score keras: 0.7444662726525472
ACC score keras: 0.7578125
-------------------------------
F1 score: 0.7625
-------------------------------
F1 score keras: 0.751936654485674
ACC score keras: 0.7625
-------------------------------
F1 score: 0.7604166666666666
-------------------------------
F1 score keras: 0.7466649643063886
ACC score keras: 0.7604166666666666
-------------------------------
F1 score: 0.7455357142857143
-------------------------------
F1 score keras: 0.7285042879604842
ACC score ker

F1 score: 0.7528935185185185
-------------------------------
F1 score keras: 0.7399904434341464
ACC score keras: 0.7528935185185185
-------------------------------
F1 score: 0.7511363636363636
-------------------------------
F1 score keras: 0.7385129837122557
ACC score keras: 0.7511363636363636
-------------------------------
F1 score: 0.7488839285714286
-------------------------------
F1 score keras: 0.7365355693205686
ACC score keras: 0.7488839285714286
-------------------------------
F1 score: 0.7494517543859649
-------------------------------
F1 score keras: 0.7373086647129967
ACC score keras: 0.7494517543859649
-------------------------------
